test model from clean samples from various speakers

https://huggingface.co/models?other=voice-activity-detection

# Hugging Face Model Set up

In [19]:
pip install -U "huggingface_hub[cli]" datasets

Note: you may need to restart the kernel to use updated packages.


In [20]:
! huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): Traceback (most recent call last):
  File "/home/codespace/.python/curr

In [ ]:
!pip install python-dotenv

In [ ]:
# check if google colaboratory is running
try:
    from google import colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    from google.colab import userdata
    HF_TOKEN = userdata.get('HF-Voizk')
else:
    # read from .env file
    import os
    from dotenv import load_dotenv
    load_dotenv()
    HF_TOKEN = os.getenv('HF_TOKEN')

# Model 1 : PyAnnote Voice activity detection


In [ ]:
pip install pyannote.audio


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from pyannote.audio import Pipeline
pipeline = Pipeline.from_pretrained("pyannote/voice-activity-detection",
                                    use_auth_token=HF_TOKEN)
output_1 = pipeline("voxceleb/TomCruiseSpeaks_sample1.wav")
output_2 = pipeline("voxceleb/TomCruiseSpeaks_sample2.wav")
output_3 = pipeline("voxceleb/TomCruiseSpeaks_sampleshort.wav")

print("Output Sample 1: ", output_1)
print("Output Sample 2: ", output_2)
print("Output Sample Short: ", output_3)


Lightning automatically upgraded your loaded checkpoint from v1.1.3 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../home/codespace/.cache/torch/pyannote/models--pyannote--segmentation/snapshots/059e96f964841d40f1a5e755bb7223f76666bba4/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.7.1, yours is 2.5.1+cu124. Bad things might happen unless you revert torch to 1.x.
Output Sample 1:  [ 00:00:00.284 -->  00:00:03.270] A SPEECH
Output Sample 2:  [ 00:00:00.233 -->  00:00:03.405] A SPEECH
Output Sample Short:  [ 00:00:00.030 -->  00:00:01.448] A SPEECH


# Model 2 : Silero VAD

In [ ]:
import torch
torch.set_num_threads(1)

model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad', model='silero_vad')
(get_speech_timestamps, _, read_audio, _, _) = utils

wav1 = read_audio('voxceleb/TomCruiseSpeaks_sample1.wav')
wav2 = read_audio('voxceleb/TomCruiseSpeaks_sample2.wav')
wav3 = read_audio('voxceleb/TomCruiseSpeaks_sampleshort.wav')

speech_timestamps1 = get_speech_timestamps(
  wav1,
  model,
  return_seconds=True,  # Return speech timestamps in seconds (default is samples)
)

print(speech_timestamps1)

speech_timestamps2 = get_speech_timestamps(
  wav2,
  model,
  return_seconds=True,  # Return speech timestamps in seconds (default is samples)
)

print(speech_timestamps2)

speech_timestamps3 = get_speech_timestamps(
  wav3,
  model,
  return_seconds=True,  # Return speech timestamps in seconds (default is samples)
)

print(speech_timestamps3)

Using cache found in /home/codespace/.cache/torch/hub/snakers4_silero-vad_master


[{'start': 0.3, 'end': 1.5}, {'start': 2.0, 'end': 3.3}]
[{'start': 0.2, 'end': 3.4}]
[{'start': 0.0, 'end': 1.3}]


# Model 3 : Wave2Vec2

In [ ]:
pip install librosa soundfile

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Let's see how to retrieve time steps for a model
from transformers import AutoTokenizer, AutoFeatureExtractor, AutoModelForCTC
from datasets import load_dataset
import datasets
import torch
import librosa

# import model, feature extractor, tokenizer
model = AutoModelForCTC.from_pretrained("facebook/wav2vec2-base-960h")
tokenizer = AutoTokenizer.from_pretrained("facebook/wav2vec2-base-960h")
feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")

def wav2vec2_processor(file_path):
    audio_input, _ = librosa.load(file_path, sr=16_000)
    # forward sample through model to get greedily predicted transcription ids    
    input_values = feature_extractor(audio_input, return_tensors="pt").input_values
    # retrieve word stamps (analogous commands for `output_char_offsets`)
    logits = model(input_values).logits[0]
    pred_ids = torch.argmax(logits, axis=-1)
    outputs = tokenizer.decode(pred_ids, output_word_offsets=True)
    # compute `time_offset` in seconds as product of downsampling ratio and sampling_rate
    time_offset = model.config.inputs_to_logits_ratio / feature_extractor.sampling_rate

    word_offsets = [
        {
            "word": d["word"],
            "start_time": round(d["start_offset"] * time_offset, 2),
            "end_time": round(d["end_offset"] * time_offset, 2),
        }
        for d in outputs.word_offsets
    ]
    return word_offsets

# test
output_1 = wav2vec2_processor("voxceleb/TomCruiseSpeaks_sample1.wav")
output_2 = wav2vec2_processor("voxceleb/TomCruiseSpeaks_sample2.wav")
output_3 = wav2vec2_processor("voxceleb/TomCruiseSpeaks_sampleshort.wav")

print("Output Sample 1: ", output_1)
print("Output Sample 2: ", output_2)
print("Output Sample Short: ", output_3)

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard

Output Sample 1:  [{'word': 'AT', 'start_time': 0.34, 'end_time': 0.38}, {'word': 'THE', 'start_time': 0.44, 'end_time': 0.54}, {'word': 'END', 'start_time': 0.62, 'end_time': 0.7}, {'word': 'OF', 'start_time': 0.78, 'end_time': 0.82}, {'word': 'IT', 'start_time': 0.92, 'end_time': 0.98}, {'word': 'YOU', 'start_time': 1.08, 'end_time': 1.2}, {'word': 'YOU', 'start_time': 2.08, 'end_time': 2.16}, {'word': 'WANT', 'start_time': 2.24, 'end_time': 2.38}, {'word': 'TO', 'start_time': 2.4, 'end_time': 2.46}, {'word': 'GO', 'start_time': 2.5, 'end_time': 2.6}, {'word': 'OUT', 'start_time': 2.74, 'end_time': 2.82}, {'word': 'AND', 'start_time': 2.88, 'end_time': 2.94}, {'word': 'YOU', 'start_time': 2.96, 'end_time': 3.04}, {'word': 'WANT', 'start_time': 3.1, 'end_time': 3.2}]
Output Sample 2:  [{'word': 'YE', 'start_time': 0.2, 'end_time': 0.3}, {'word': 'I', 'start_time': 0.42, 'end_time': 0.44}, {'word': 'LOVE', 'start_time': 0.54, 'end_time': 0.7}, {'word': 'THESE', 'start_time': 0.74, 'end

#  Model 4 : WebRTC

The WebRTC VAD only accepts 16-bit mono PCM audio, sampled at 8000, 16000, 32000 or 48000 Hz. A frame must be either 10, 20, or 30 ms in duration:

In [ ]:
pip install webrtcvad


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp312-cp312-linux_x86_64.whl size=92349 sha256=c406ba4441f97b02c648eba2fd88e0108b0a95d92980e83e88f782f64a6c4602
  Stored in directory: /home/codespace/.cache/pip/wheels/1e/d3/95/680fa3b16848f1a58d2edaed34c496224c89a9bc63e17b3614
Successfully built webrtcvad
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import collections
import contextlib
import sys
import wave

import webrtcvad


def read_wave(path):
    """Reads a .wav file.

    Takes the path, and returns (PCM audio data, sample rate).
    """
    with contextlib.closing(wave.open(path, 'rb')) as wf:
        num_channels = wf.getnchannels()
        assert num_channels == 1
        sample_width = wf.getsampwidth()
        assert sample_width == 2
        sample_rate = wf.getframerate()
        assert sample_rate in (8000, 16000, 32000, 48000)
        pcm_data = wf.readframes(wf.getnframes())
        return pcm_data, sample_rate


def write_wave(path, audio, sample_rate):
    """Writes a .wav file.

    Takes path, PCM audio data, and sample rate.
    """
    with contextlib.closing(wave.open(path, 'wb')) as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(sample_rate)
        wf.writeframes(audio)


class Frame(object):
    """Represents a "frame" of audio data."""
    def __init__(self, bytes, timestamp, duration):
        self.bytes = bytes
        self.timestamp = timestamp
        self.duration = duration


def frame_generator(frame_duration_ms, audio, sample_rate):
    """Generates audio frames from PCM audio data.

    Takes the desired frame duration in milliseconds, the PCM data, and
    the sample rate.

    Yields Frames of the requested duration.
    """
    n = int(sample_rate * (frame_duration_ms / 1000.0) * 2)
    offset = 0
    timestamp = 0.0
    duration = (float(n) / sample_rate) / 2.0
    while offset + n < len(audio):
        yield Frame(audio[offset:offset + n], timestamp, duration)
        timestamp += duration
        offset += n


def vad_collector(sample_rate, frame_duration_ms,
                  padding_duration_ms, vad, frames):
    """Filters out non-voiced audio frames.

    Given a webrtcvad.Vad and a source of audio frames, yields only
    the voiced audio.

    Uses a padded, sliding window algorithm over the audio frames.
    When more than 90% of the frames in the window are voiced (as
    reported by the VAD), the collector triggers and begins yielding
    audio frames. Then the collector waits until 90% of the frames in
    the window are unvoiced to detrigger.

    The window is padded at the front and back to provide a small
    amount of silence or the beginnings/endings of speech around the
    voiced frames.

    Arguments:

    sample_rate - The audio sample rate, in Hz.
    frame_duration_ms - The frame duration in milliseconds.
    padding_duration_ms - The amount to pad the window, in milliseconds.
    vad - An instance of webrtcvad.Vad.
    frames - a source of audio frames (sequence or generator).

    Returns: A generator that yields PCM audio data.
    """
    num_padding_frames = int(padding_duration_ms / frame_duration_ms)
    # We use a deque for our sliding window/ring buffer.
    ring_buffer = collections.deque(maxlen=num_padding_frames)
    # We have two states: TRIGGERED and NOTTRIGGERED. We start in the
    # NOTTRIGGERED state.
    triggered = False

    voiced_frames = []
    for frame in frames:
        is_speech = vad.is_speech(frame.bytes, sample_rate)

        sys.stdout.write('1' if is_speech else '0')
        if not triggered:
            ring_buffer.append((frame, is_speech))
            num_voiced = len([f for f, speech in ring_buffer if speech])
            # If we're NOTTRIGGERED and more than 90% of the frames in
            # the ring buffer are voiced frames, then enter the
            # TRIGGERED state.
            if num_voiced > 0.9 * ring_buffer.maxlen:
                triggered = True
                sys.stdout.write('+(%s)' % (ring_buffer[0][0].timestamp,))
                # We want to yield all the audio we see from now until
                # we are NOTTRIGGERED, but we have to start with the
                # audio that's already in the ring buffer.
                for f, s in ring_buffer:
                    voiced_frames.append(f)
                ring_buffer.clear()
        else:
            # We're in the TRIGGERED state, so collect the audio data
            # and add it to the ring buffer.
            voiced_frames.append(frame)
            ring_buffer.append((frame, is_speech))
            num_unvoiced = len([f for f, speech in ring_buffer if not speech])
            # If more than 90% of the frames in the ring buffer are
            # unvoiced, then enter NOTTRIGGERED and yield whatever
            # audio we've collected.
            if num_unvoiced > 0.9 * ring_buffer.maxlen:
                sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
                triggered = False
                yield b''.join([f.bytes for f in voiced_frames])
                ring_buffer.clear()
                voiced_frames = []
    if triggered:
        sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
    sys.stdout.write('\n')
    # If we have any leftover voiced audio when we run out of input,
    # yield it.
    if voiced_frames:
        yield b''.join([f.bytes for f in voiced_frames])


INPUT1 = "voxceleb/TomCruiseSpeaks_sample1.wav"
INPUT2 = "voxceleb/TomCruiseSpeaks_sample2.wav"
INPUT3 = "voxceleb/TomCruiseSpeaks_sampleshort.wav"

audio, sample_rate = read_wave("voxceleb/TomCruiseSpeaks_sample1.wav")
vad = webrtcvad.Vad()
frames = frame_generator(30, audio, sample_rate)
frames = list(frames)
segments = vad_collector(sample_rate, 30, 300, vad, frames)
for i, segment in enumerate(segments):
    path = 'chunk-%002d.wav' % (i,)
    print(' Writing %s' % (path,))
    write_wave(path, segment, sample_rate)

AssertionError: 

# Whisper Tiny

In [ ]:
!sudo apt-get update && sudo apt-get install -y ffmpeg

In [ ]:
!pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 14.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.7 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803321 sha256=727db80b46c9566f72100049aecb3e5e2d426d41fb1e134ec8d669079d6203e0
  Stored in directory: /home/codespace/.cache/pip/wheels/7c/f5/6f/92094c35416f9397abb86b23cfe72fb255a3013012f983136d
Successfully built openai-whisper
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import whisper

# Load the Whisper Tiny model
model = whisper.load_model("tiny")

# Function to perform voice activity detection
def whisper_vad(file_path):
    result = model.transcribe(file_path)
    segments = result['segments']
    return segments

# Test the function
segments_1 = whisper_vad("voxceleb/TomCruiseSpeaks_sample1.wav")
segments_2 = whisper_vad("voxceleb/TomCruiseSpeaks_sample2.wav")
segments_3 = whisper_vad("voxceleb/TomCruiseSpeaks_sampleshort.wav")

print("Segments Sample 1: ", segments_1)
print("Segments Sample 2: ", segments_2)
print("Segments Sample Short: ", segments_3)

/usr/local/python/3.12.1/lib/python3.12/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)

Segments Sample 1:  [{'id': 0, 'seek': 0, 'start': 0.0, 'end': 3.3200000000000003, 'text': ' At the end of it you you want to go out and you want to', 'tokens': [50364, 1711, 264, 917, 295, 309, 291, 291, 528, 281, 352, 484, 293, 291, 528, 281, 50530], 'temperature': 0.0, 'avg_logprob': -0.31429534488254124, 'compression_ratio': 1.1458333333333333, 'no_speech_prob': 0.2468426525592804}]
Segments Sample 2:  [{'id': 0, 'seek': 0, 'start': 0.0, 'end': 3.36, 'text': " You know, I love these ceremonies. I remember there's a little kid I used to", 'tokens': [50364, 509, 458, 11, 286, 959, 613, 36176, 13, 286, 1604, 456, 311, 257, 707, 1636, 286, 1143, 281, 50532], 'temperature': 0.0, 'avg_logprob': -0.39154079982212614, 'compression_ratio': 1.0133333333333334, 'no_speech_prob': 0.037378426641225815}]
Segments Sample Short:  [{'id': 0, 'seek': 0, 'start': 0.0, 'end': 2.0, 'text': ' want to embrace life.', 'tokens': [50364, 528, 281, 14038, 993, 13, 50464], 'temperature': 0.0, 'avg_logprob': -